# AlexNet
This notebook was motivated by
[1] Alex Krizhevsky, Ilya Sutskever and Geoffrey E Hinton. ‘ImageNet Classification with Deep Convolutional Neural Networks’. In: (2012). url: https://proceedings.neurips.cc/paper/2012/file/c 399862d3b9d6b76c8436e924a68c45b-Paper.pdf.

https://papers.nips.cc/paper/2012/hash/c399862d3b9d6b76c8436e924a68c45b-Abstract.html

Implementation: Oleh Bakumenko, Univerity of Duisburg-Essen



# Imports

In [ ]:
import sys
sys.path.append("../") # Otherwise, import from the local folder's parent folder, where your stuff lives.

import os
import numpy as np
import time
import matplotlib.pyplot as plt
import torch, torch.nn as nn
import torchvision, torchvision.transforms as tt
from torch.multiprocessing import Manager
torch.multiprocessing.set_sharing_strategy("file_system")
from pathlib import Path

from utility import utils as uu
from utility.eval import evaluate_classifier_model
from utility.confusion_matrix import calculate_confusion_matrix

from utility.trainLoopClassifier import *
from utility.plotImageModel import *

# Data augmentations

In [ ]:
data_augments = torchvision.transforms.Compose([ 
    torchvision.transforms.RandomHorizontalFlip(p = .5),
    torchvision.transforms.RandomVerticalFlip(p = .5),
    torchvision.transforms.ColorJitter(brightness=(0.5,1.5), contrast=(1), hue=(-0.1,0.1)),
    torchvision.transforms.RandomCrop((224, 224)), 
    ])


Load the dataset from utils

In [ ]:
cur_path = Path("plots_and_graphs.ipynb")
parent_dir = cur_path.parent.absolute()
masterThesis_folder = str(parent_dir.parent.absolute())+'/'
data_dir = masterThesis_folder+"data/Clean_LiTS/"

cache_me = False
if cache_me is True:
    cache_mgr = Manager()
    cache_mgr.data = cache_mgr.dict()
    cache_mgr.cached = cache_mgr.dict()
    for k in ["train", "val", "test"]:
        cache_mgr.data[k] = cache_mgr.dict()
        cache_mgr.cached[k] = False
# function from utils, credit: Institute for Artificial Intelligence in Medicine. url: https://mml.ikim.nrw/
# dataset outputs a tensor image (dimensions [1,256,256]) and a tensor target (0, 1 or 2)

ds = uu.LiTS_Classification_Dataset(
    data_dir=data_dir,
    transforms=data_augments,
    verbose=True,
    cache_data=cache_me,
    cache_mgr=(cache_mgr if cache_me is True else None),
    debug=True,
)

# Hyperparameters

In [ ]:
# Default settings
batch_size = 32
epochs = 50
device = ("cuda" if torch.cuda.is_available() else "cpu")
time_me  = True

The `torch.utils.data.DataLoader` is a utility class in PyTorch that makes the loading and batching of data for training purposes faster. It simplifies the process by allowing us to specify the dataset, batch size (often 32), and whether the data should be shuffled before each epoch. Additionally, there are other parameters available to further customize the data loading process.

In [ ]:
# Dataloader
dl = torch.utils.data.DataLoader(
    dataset = ds, 
    batch_size = batch_size, 
    num_workers = 4, 
    shuffle = True, 
    drop_last = False, 
    pin_memory = True,
    persistent_workers = (not cache_me),
    prefetch_factor = 1
    )

# AlexNet

AlexNet is a deep convolutional neural network architecture that was introduced in 2012 by Alex Krizhevsky, Ilya Sutskever, and Geoffrey Hinton. It was one of the first successful models to use deep convolutional neural networks for image classification and won the ImageNet Large Scale Visual Recognition Challenge (ILSVRC) in 2012. AlexNet consists of eight layers, including five convolutional layers and three fully connected layers, and uses ReLU activation functions and pooling layers to reduce the dimensionality of the input data.

Some bullet points:
1. ReLU activation function instead of tanh.
2. Using dropout to reduce overfitting.
3. Main idea: convolution followed by max pooling, and stacking of these layers; using dropout for fully connected layers.
4. Using parallel computations on multiple GPUs (not included here).
5. Local Response Normalization - not used in future networks; see torch.nn.LocalResponseNorm.

Overall architecture and channel sizes can be found in [1].

In [ ]:
# AlexNet Class
#       - constructs a convolutional neuronal network as described in [1]
# Input:    Tensor: [Batch,1,Height,Width]
# Output:   Tensor: [Batch,3]
import torch.nn.functional as F

class AlexNetMLMed(torch.nn.Module):
    def __init__(self):
        super(AlexNetMLMed, self).__init__()
        self.conv1 = torch.nn.Conv2d(in_channels = 1, out_channels = 96, kernel_size = (11,11), stride = (4,4), padding=1)        
        self.pool1 = torch.nn.MaxPool2d(kernel_size = 3, stride = 2)
        self.responseNorm = torch.nn.LocalResponseNorm(5)
        self.conv2 = torch.nn.Conv2d(in_channels = 96, out_channels = 256, kernel_size = (5, 5), stride = (1,1),padding=1)
        self.pool2 = torch.nn.MaxPool2d(kernel_size = 3, stride = 2)

        self.conv3 = torch.nn.Conv2d(in_channels = 256, out_channels = 384, kernel_size = (3, 3), stride = (1,1),padding=1)
        
        self.conv4 = torch.nn.Conv2d(in_channels = 384, out_channels = 384, kernel_size = (3, 3), stride = (1,1),padding=1)

        self.conv5 = torch.nn.Conv2d(in_channels = 384, out_channels = 256, kernel_size = (3, 3), stride = (1,1),padding=1)

        self.pool4 = torch.nn.MaxPool2d(kernel_size = 3, stride = 2)
        
        self.dropout = torch.nn.Dropout(p=0.5) 

        self.fc1 = nn.Linear(9216,4096)
        self.fc1_1 = nn.Linear(6400,4096)

        self.fc2 = nn.Linear(4096,4096)
        self.fc3 = nn.Linear(4096,3)

        


    def forward(self, x):
        out = F.relu(self.conv1(x))
        out = self.pool1(out)
        out = self.responseNorm(out)

        out = F.relu(self.conv2(out))
        out = self.pool2(out)
        out = self.responseNorm(out)


        out = F.relu(self.conv3(out))
        out = F.relu(self.conv4(out))
        out = F.relu(self.conv5(out))
        out = self.pool4(out)


        out = out.flatten(start_dim=1)

        out = self.dropout(out)
        out = F.relu(self.fc1_1(out))
        out = self.dropout(out)
        out = F.relu(self.fc2(out))
        out = self.dropout(out)
        out = F.relu(self.fc3(out))
        
        return out

In [ ]:
model = AlexNetMLMed()
model = model.to(device)

In [ ]:
for step, (data, targets) in enumerate(dl):
    data, targets = data.to(device), targets.to(device)
    if step ==1:
        break
model(data).shape

In [ ]:
learning_rate = 1e-5
run_name = "AlexNet_fixed_time_lre5"

In [ ]:
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)
criterion = nn.CrossEntropyLoss()

In [ ]:
uu.csv_logger(
        logfile = f"../logs/{run_name}_hyperparams.csv",
        content = {"learning_rate": learning_rate, "batch_size": batch_size, "epochs": epochs},
        first= True,
        overwrite= True)

In [ ]:
mod_step = 5000
wantToPrint = False
stop_bool = False
eval_test_10_min = False
eval_test_15_min = False
eval_test_20_min = False
skip_test_10_min = False
skip_test_15_min = False
skip_test_20_min = False

Modified training loop. The starting time is saved. Time elapsed is calcualated at the beginning of the epoch and if it is > 10, 15 or 20 minutes the boolean flag for the test evaluation is set to True; Howewer it is important that the test evaluation only one time happens, therefore after the calculation the boolen flag for skip is set to true. This ensure that the test evaluation happens only one time.

During the test evaluation the dataset mode is switched to "test", model is turned to the evaluation mode and the test accuracy, loss, confusion matrix and per class accuracy is calculated and saved.
The same procedure repeted 3 times for each time step.

In [ ]:
train_start = time.time()

num_steps = len(ds.file_names['train'])//batch_size

for epoch in range(epochs):
    time_elapsed = time.time() - train_start
    print(f"Time_elapsed: {time_elapsed/60 :.2f} min")
    if time_elapsed > 10*60:
        eval_test_10_min = True
    if time_elapsed > 15*60:
        eval_test_15_min = True
    if time_elapsed > 20*60:
        eval_test_20_min = True

    if eval_test_10_min and not skip_test_10_min:
        print('Evaluate after first 10 min')
        with torch.no_grad():
            ds.set_mode("test")
            model.eval()
            test_accuracy, avg_test_loss = evaluate_classifier_model(model = model, dataloader = dl, device = device)
            confusion_matrix, acc = calculate_confusion_matrix(model=model, dataloader=dl, device=device)
            torch.save(confusion_matrix, f = 'confusion_matr_' + run_name+ '_10min' + '.pt')
            print(f"Evaluate after first 10 min: Test Loss: {avg_test_loss:.4f}\t Test Accuracy: {test_accuracy:.4f}, Confusion Matrix: \n{confusion_matrix}, Per-class Accuracy: {acc}")
            uu.csv_logger(
                logfile = f"../logs/{run_name}_test.csv",
                content = {"epoch": epoch,"test_phase": 1, "test_loss": avg_test_loss, "test_accuracy": test_accuracy, "time_elapsed": time_elapsed})
        skip_test_10_min = True

    if eval_test_15_min and not skip_test_15_min:
        print('Evaluate after first 15 min')
        with torch.no_grad():
            ds.set_mode("test")
            model.eval()
            test_accuracy, avg_test_loss = evaluate_classifier_model(model = model, dataloader = dl, device = device)
            confusion_matrix, acc = calculate_confusion_matrix(model=model, dataloader=dl, device=device)
            torch.save(confusion_matrix, f = 'confusion_matr_' + run_name+ '_15min' + '.pt')
            print(f"Evaluate after first 15 min: Test Loss: {avg_test_loss:.4f}\t Test Accuracy: {test_accuracy:.4f}, Confusion Matrix: \n{confusion_matrix}, Per-class Accuracy: {acc}")
            uu.csv_logger(
                logfile = f"../logs/{run_name}_test.csv",
                content = {"epoch": epoch,"test_phase": 2, "test_loss": avg_test_loss, "test_accuracy": test_accuracy, "time_elapsed": time_elapsed})
        skip_test_15_min = True

    if eval_test_20_min and not skip_test_20_min:
        print('Evaluate after first 20 min')
        with torch.no_grad():
            ds.set_mode("test")
            model.eval()
            test_accuracy, avg_test_loss = evaluate_classifier_model(model = model, dataloader = dl, device = device)
            confusion_matrix, acc = calculate_confusion_matrix(model=model, dataloader=dl, device=device)
            torch.save(confusion_matrix, f = 'confusion_matr_' + run_name+ '_20min' + '.pt')
            print(f"Evaluate after first 20 min: Test Loss: {avg_test_loss:.4f}\t Test Accuracy: {test_accuracy:.4f}, Confusion Matrix: \n{confusion_matrix}, Per-class Accuracy: {acc}")
            uu.csv_logger(
                logfile = f"../logs/{run_name}_test.csv",
                content = {"epoch": epoch,"test_phase": 3, "test_loss": avg_test_loss, "test_accuracy": test_accuracy, "time_elapsed": time_elapsed})
        skip_test_20_min = True

        break

    # Go to train mode
    ds.set_mode("train")
    model.train()

    # Train loop
    for step, (data, targets) in enumerate(dl):

        # Manually drop last batch (this is for example relevant with BatchNorm)
        if step == num_steps - 1 and (epoch > 0 or ds.cache_data is False):
            continue

        # Train loop: Zero gradients, forward step, evaluate, log, backward step
        optimizer.zero_grad()
        data, targets = data.to(device), targets.to(device)
        predictions = model(data)
        loss = criterion(predictions, targets)
        loss.backward()
        optimizer.step()

    # Go to eval mode
    ds.set_mode("val")
    model.eval()

    # Validation loop
    val_accuracy, avg_val_loss = evaluate_classifier_model(model = model, dataloader = dl, device = device)
    print(f"Epoch [{epoch+1}/{epochs}]\t Val Loss: {avg_val_loss:.4f}\t Val Accuracy: {val_accuracy:.4f}")
    uu.csv_logger(
        logfile = f"../logs/{run_name}_val.csv",
        content = {"epoch": epoch, "val_loss": avg_val_loss, "val_accuracy": val_accuracy},
        first = (epoch == 0),
        overwrite = (epoch == 0)
            )

---

In [ ]:
run_name = "AlexNet_fixed_time_lre4"
learning_rate = 1e-4

In [ ]:
del model

In [ ]:
model = AlexNetMLMed()
model = model.to(device)

In [ ]:
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)
criterion = nn.CrossEntropyLoss()

In [ ]:
stop_bool = False
eval_test_10_min = False
eval_test_15_min = False
eval_test_20_min = False
skip_test_10_min = False
skip_test_15_min = False
skip_test_20_min = False

In [ ]:
train_start = time.time()

num_steps = len(ds.file_names['train'])//batch_size

for epoch in range(epochs):
    time_elapsed = time.time() - train_start
    print(f"Time_elapsed: {time_elapsed/60 :.2f} min")
    if time_elapsed > 10*60:
        eval_test_10_min = True
    if time_elapsed > 15*60:
        eval_test_15_min = True
    if time_elapsed > 20*60:
        eval_test_20_min = True

    if eval_test_10_min and not skip_test_10_min:
        print('Evaluate after first 10 min')
        with torch.no_grad():
            ds.set_mode("test")
            model.eval()
            test_accuracy, avg_test_loss = evaluate_classifier_model(model = model, dataloader = dl, device = device)
            confusion_matrix, acc = calculate_confusion_matrix(model=model, dataloader=dl, device=device)
            torch.save(confusion_matrix, f = 'confusion_matr_' + run_name+ '_10min' + '.pt')
            print(f"Evaluate after first 10 min: Test Loss: {avg_test_loss:.4f}\t Test Accuracy: {test_accuracy:.4f}, Confusion Matrix: \n{confusion_matrix}, Per-class Accuracy: {acc}")
            uu.csv_logger(
                logfile = f"../logs/{run_name}_test.csv",
                content = {"epoch": epoch,"test_phase": 1, "test_loss": avg_test_loss, "test_accuracy": test_accuracy, "time_elapsed": time_elapsed})
        skip_test_10_min = True

    if eval_test_15_min and not skip_test_15_min:
        print('Evaluate after first 15 min')
        with torch.no_grad():
            ds.set_mode("test")
            model.eval()
            test_accuracy, avg_test_loss = evaluate_classifier_model(model = model, dataloader = dl, device = device)
            confusion_matrix, acc = calculate_confusion_matrix(model=model, dataloader=dl, device=device)
            torch.save(confusion_matrix, f = 'confusion_matr_' + run_name+ '_15min' + '.pt')
            print(f"Evaluate after first 15 min: Test Loss: {avg_test_loss:.4f}\t Test Accuracy: {test_accuracy:.4f}, Confusion Matrix: \n{confusion_matrix}, Per-class Accuracy: {acc}")
            uu.csv_logger(
                logfile = f"../logs/{run_name}_test.csv",
                content = {"epoch": epoch,"test_phase": 2, "test_loss": avg_test_loss, "test_accuracy": test_accuracy, "time_elapsed": time_elapsed})
        skip_test_15_min = True

    if eval_test_20_min and not skip_test_20_min:
        print('Evaluate after first 20 min')
        with torch.no_grad():
            ds.set_mode("test")
            model.eval()
            test_accuracy, avg_test_loss = evaluate_classifier_model(model = model, dataloader = dl, device = device)
            confusion_matrix, acc = calculate_confusion_matrix(model=model, dataloader=dl, device=device)
            torch.save(confusion_matrix, f = 'confusion_matr_' + run_name+ '_20min' + '.pt')
            print(f"Evaluate after first 20 min: Test Loss: {avg_test_loss:.4f}\t Test Accuracy: {test_accuracy:.4f}, Confusion Matrix: \n{confusion_matrix}, Per-class Accuracy: {acc}")
            uu.csv_logger(
                logfile = f"../logs/{run_name}_test.csv",
                content = {"epoch": epoch,"test_phase": 3, "test_loss": avg_test_loss, "test_accuracy": test_accuracy, "time_elapsed": time_elapsed})
        break

    # Go to train mode
    ds.set_mode("train")
    model.train()

    # Train loop
    for step, (data, targets) in enumerate(dl):

        # Manually drop last batch (this is for example relevant with BatchNorm)
        if step == num_steps - 1 and (epoch > 0 or ds.cache_data is False):
            continue

        # Train loop: Zero gradients, forward step, evaluate, log, backward step
        optimizer.zero_grad()
        data, targets = data.to(device), targets.to(device)
        predictions = model(data)
        loss = criterion(predictions, targets)
        loss.backward()
        optimizer.step()

    # Go to eval mode
    ds.set_mode("val")
    model.eval()

    # Validation loop
    val_accuracy, avg_val_loss = evaluate_classifier_model(model = model, dataloader = dl, device = device)
    print(f"Epoch [{epoch+1}/{epochs}]\t Val Loss: {avg_val_loss:.4f}\t Val Accuracy: {val_accuracy:.4f}")
    uu.csv_logger(
        logfile = f"../logs/{run_name}_val.csv",
        content = {"epoch": epoch, "val_loss": avg_val_loss, "val_accuracy": val_accuracy},
        first = (epoch == 0),
        overwrite = (epoch == 0)
            )

---

In [ ]:
run_name = "AlexNet_fixed_time_lre3"
learning_rate = 1e-3

In [ ]:
del model

In [ ]:
model = AlexNetMLMed()
model = model.to(device)

In [ ]:
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)
criterion = nn.CrossEntropyLoss()

In [ ]:
stop_bool = False
eval_test_10_min = False
eval_test_15_min = False
eval_test_20_min = False
skip_test_10_min = False
skip_test_15_min = False
skip_test_20_min = False

In [ ]:
train_start = time.time()

num_steps = len(ds.file_names['train'])//batch_size

for epoch in range(epochs):
    time_elapsed = time.time() - train_start
    print(f"Time_elapsed: {time_elapsed/60 :.2f} min")
    if time_elapsed > 10*60:
        eval_test_10_min = True
    if time_elapsed > 15*60:
        eval_test_15_min = True
    if time_elapsed > 20*60:
        eval_test_20_min = True

    if eval_test_10_min and not skip_test_10_min:
        print('Evaluate after first 10 min')
        with torch.no_grad():
            ds.set_mode("test")
            model.eval()
            test_accuracy, avg_test_loss = evaluate_classifier_model(model = model, dataloader = dl, device = device)
            confusion_matrix, acc = calculate_confusion_matrix(model=model, dataloader=dl, device=device)
            torch.save(confusion_matrix, f = 'confusion_matr_' + run_name+ '_10min' + '.pt')
            print(f"Evaluate after first 10 min: Test Loss: {avg_test_loss:.4f}\t Test Accuracy: {test_accuracy:.4f}, Confusion Matrix: \n{confusion_matrix}, Per-class Accuracy: {acc}")
            uu.csv_logger(
                logfile = f"../logs/{run_name}_test.csv",
                content = {"epoch": epoch,"test_phase": 1, "test_loss": avg_test_loss, "test_accuracy": test_accuracy, "time_elapsed": time_elapsed})
        skip_test_10_min = True

    if eval_test_15_min and not skip_test_15_min:
        print('Evaluate after first 15 min')
        with torch.no_grad():
            ds.set_mode("test")
            model.eval()
            test_accuracy, avg_test_loss = evaluate_classifier_model(model = model, dataloader = dl, device = device)
            confusion_matrix, acc = calculate_confusion_matrix(model=model, dataloader=dl, device=device)
            torch.save(confusion_matrix, f = 'confusion_matr_' + run_name+ '_15min' + '.pt')
            print(f"Evaluate after first 15 min: Test Loss: {avg_test_loss:.4f}\t Test Accuracy: {test_accuracy:.4f}, Confusion Matrix: \n{confusion_matrix}, Per-class Accuracy: {acc}")
            uu.csv_logger(
                logfile = f"../logs/{run_name}_test.csv",
                content = {"epoch": epoch,"test_phase": 2, "test_loss": avg_test_loss, "test_accuracy": test_accuracy, "time_elapsed": time_elapsed})
        skip_test_15_min = True

    if eval_test_20_min and not skip_test_20_min:
        print('Evaluate after first 20 min')
        with torch.no_grad():
            ds.set_mode("test")
            model.eval()
            test_accuracy, avg_test_loss = evaluate_classifier_model(model = model, dataloader = dl, device = device)
            confusion_matrix, acc = calculate_confusion_matrix(model=model, dataloader=dl, device=device)
            torch.save(confusion_matrix, f = 'confusion_matr_' + run_name+ '_20min' + '.pt')
            print(f"Evaluate after first 20 min: Test Loss: {avg_test_loss:.4f}\t Test Accuracy: {test_accuracy:.4f}, Confusion Matrix: \n{confusion_matrix}, Per-class Accuracy: {acc}")
            uu.csv_logger(
                logfile = f"../logs/{run_name}_test.csv",
                content = {"epoch": epoch,"test_phase": 3, "test_loss": avg_test_loss, "test_accuracy": test_accuracy, "time_elapsed": time_elapsed})
        break

    # Go to train mode
    ds.set_mode("train")
    model.train()

    # Train loop
    for step, (data, targets) in enumerate(dl):

        # Manually drop last batch (this is for example relevant with BatchNorm)
        if step == num_steps - 1 and (epoch > 0 or ds.cache_data is False):
            continue

        # Train loop: Zero gradients, forward step, evaluate, log, backward step
        optimizer.zero_grad()
        data, targets = data.to(device), targets.to(device)
        predictions = model(data)
        loss = criterion(predictions, targets)
        loss.backward()
        optimizer.step()

    # Go to eval mode
    ds.set_mode("val")
    model.eval()

    # Validation loop
    val_accuracy, avg_val_loss = evaluate_classifier_model(model = model, dataloader = dl, device = device)
    print(f"Epoch [{epoch+1}/{epochs}]\t Val Loss: {avg_val_loss:.4f}\t Val Accuracy: {val_accuracy:.4f}")
    uu.csv_logger(
        logfile = f"../logs/{run_name}_val.csv",
        content = {"epoch": epoch, "val_loss": avg_val_loss, "val_accuracy": val_accuracy},
        first = (epoch == 0),
        overwrite = (epoch == 0)
            )